# Python Package Import

In [1]:
import os
import json
import fnmatch
import glob
import bids
import shutil
import multiprocessing
from bids import BIDSLayout
from joblib import Parallel, delayed

# BIDS Data Organisation

Use dcm2niix to convert from DICOM to NIFTI and place the NIFTI files in a BIDS structure.

In [2]:
!dcm2niix -v

Chris Rorden's dcm2niiX version v1.0.20181125  (JP2:OpenJPEG) (JP-LS:CharLS) Clang8.1.0 (64-bit MacOS)
 Error: invalid option '-v (null)'
Example output filename: '/sub-23_ses-4_acq-lr_dwi.nii.gz'


In [4]:
# create subject file structure
for tp in ["1","2", "3", "4"]: # do timepoints 2:4
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-2].split("_")[0]

        os.makedirs(niidir+"/sub-"+subj[1:]+"/ses-"+tp+"/anat",exist_ok=True)
        os.makedirs(niidir+"/sub-"+subj[1:]+"/ses-"+tp+"/dwi",exist_ok=True)
        os.makedirs(niidir+"/sub-"+subj[1:]+"/ses-"+tp+"/func",exist_ok=True)


In [6]:
# Import MPR (T1) for timepoint 1
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1
dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

#MPR1
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR1*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-1_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)

#MPR2
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR2*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-2_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)

#MPR3 - only a few subjects have a third T1!
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR3*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-3_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)


dcm2niix -z y -f sub-01_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A01_83227/CRANE_BREBIS_20180319_084902_656000/T1_MPR1_0009
dcm2niix -z y -f sub-02_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A02_83256/CRANE_BREBIS_20180319_103637_359000/T1_MPR1_0015
dcm2niix -z y -f sub-03_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A03_83262/CRANE_BREBIS_20180319_120838_640000/T1_MPR1_0016
dcm2niix -z y -f sub-04_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A04_83286/CRANE_BREBIS_20180319_134850_156000/T1_MPR1_0015
dcm2niix -z y -f sub-05_ses-1_run-1_T1w -o /

In [ ]:
# The file names for both T1 runs were different for subject 4. 
# Actually the T1s were re-ran to improve them (see the README.txt in the dicom folder).
# So it is necessary to run this cell to correct it.
!rm /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/anat/*
!dcm2niix -z y -f sub-04_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A04_83286/CRANE_BREBIS_20180319_134850_156000/T1_MPR_TEST_CORO_0019
!dcm2niix -z y -f sub-04_ses-1_run-2_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A04_83286/CRANE_BREBIS_20180319_134850_156000/T1_MPR_TEST_CORO2_0023

# For subject 5 the T1s were re-ran to improve them (see the README.txt in the dicom folder).
# So it is necessary to run this cell to correct it.
!dcm2niix -z y -f sub-05_ses-1_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-05/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A05_83278/CRANE_BREBIS_20180320_080450_328000/T1_MPR2_CORO_1NEX_0021
!dcm2niix -z y -f sub-05_ses-1_run-2_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-05/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A05_83278/CRANE_BREBIS_20180320_080450_328000/T1_MPR1_CORO_2NEX_0017

# There was a 'testing' run 3 for subject 13 that needs to be removed
!rm /Volumes/data/prebiostress/data/sourcedata/nifti/sub-13/ses-1/anat/*run-3_T1w*

In [8]:
# Import SPC (T2) for timepoint 1
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1
dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

#SPC1
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC1*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-1_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)

#SPC2
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC2*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-2_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)

#SPC3 - only a few subjects have a third T2!
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC3*')
    datadir = ''.join(datadir[0:1])
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-3_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
    print(cmd)
    os.system(cmd)


dcm2niix -z y -f sub-01_ses-1_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A01_83227/CRANE_BREBIS_20180319_084902_656000/T2_SPC1_0011
dcm2niix -z y -f sub-02_ses-1_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A02_83256/CRANE_BREBIS_20180319_103637_359000/T2_SPC1_0017
dcm2niix -z y -f sub-03_ses-1_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A03_83262/CRANE_BREBIS_20180319_120838_640000/T2_SPC1_0018
dcm2niix -z y -f sub-04_ses-1_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A04_83286/CRANE_BREBIS_20180319_134850_156000/T2_SPC1_0017
dcm2niix -z y -f sub-05_ses-1_run-1_T2w -o /

In [5]:
# Import DWI for timepoint 1
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1
dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

#RL
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_RL_0*')
    datadir = ''.join(datadir)
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-rl_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
    print(cmd)
    os.system(cmd)
#LR
for fname in listdir_nohidden(dcmdir):
    subj = fname.split("_")[-2].split("_")[0]

    datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_LR_0*')
    datadir = ''.join(datadir)
    cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-lr_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
    print(cmd)
    os.system(cmd)


dcm2niix -z y -f sub-01_ses-1_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-1/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A01_83227/CRANE_BREBIS_20180319_084902_656000/DWI_MB3_DIR64_RL_0021
dcm2niix -z y -f sub-02_ses-1_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-1/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A02_83256/CRANE_BREBIS_20180319_103637_359000/DWI_MB3_DIR64_RL_0003
dcm2niix -z y -f sub-03_ses-1_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-1/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A03_83262/CRANE_BREBIS_20180319_120838_640000/DWI_MB3_DIR64_RL_0004
dcm2niix -z y -f sub-04_ses-1_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-1/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint1/OV2018RN1_A04_83286/CRANE_BREBIS_20180319_134850_156000/DWI_MB3_DIR64_RL_0003
dcm2niix

In [3]:
# Import MPR (T1) for timepoints 2,3,4
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

for tp in ["2", "3", "4"]: # do timepoints 2:4
    dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp
    
    #MPR1
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]
    
        datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR1*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-1_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)

    #MPR2
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR2*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-2_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)

    #MPR3 - only a few subjects have a third T1!
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/T1_MPR3*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-3_T1w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)
    

dcm2niix -z y -f sub-01_ses-2_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A01_P02B_83227/CRANE_BREBIS_20180413_072122_500000/T1_MPR1_CORO_1NEX_0017
dcm2niix -z y -f sub-02_ses-2_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A02_P02B_83256/CRANE_BREBIS_20180413_084039_890000/T1_MPR1_CORO_1NEX_0017
dcm2niix -z y -f sub-03_ses-2_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A03_P02B_83262/CRANE_BREBIS_20180413_100328_734000/T1_MPR1_CORO_1NEX_0017
dcm2niix -z y -f sub-04_ses-2_run-1_T1w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A04_P02B_83286/CRANE_BREBIS_20180413_111803_203000/T1_MPR1

In [4]:
# Import SPC (T2) for timepoints 2,3,4
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

for tp in ["2", "3", "4"]: # do timepoints 2:4
    dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp
    
    #SPC1
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC1*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-1_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)

    #SPC2
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC2*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-2_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)

    #SPC3 - only a few subjects have a third T2!
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/T2_SPC3*')
        datadir = ''.join(datadir[0:1])
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_run-3_T2w " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/anat " +datadir
        print(cmd)
        os.system(cmd)

dcm2niix -z y -f sub-01_ses-2_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A01_P02B_83227/CRANE_BREBIS_20180413_072122_500000/T2_SPC1_0015
dcm2niix -z y -f sub-02_ses-2_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A02_P02B_83256/CRANE_BREBIS_20180413_084039_890000/T2_SPC1_0015
dcm2niix -z y -f sub-03_ses-2_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A03_P02B_83262/CRANE_BREBIS_20180413_100328_734000/T2_SPC1_0015
dcm2niix -z y -f sub-04_ses-2_run-1_T2w -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-2/anat /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A04_P02B_83286/CRANE_BREBIS_20180413_111803_203000/T2_SPC1_0015
dcm2niix -z y -f sub-05_

In [2]:
# Import DWI for timepoints 2,3,4
# define function that lists all files except hidden files
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

toplvl = "/Volumes/data/prebiostress/data"
niidir = toplvl + "/sourcedata/nifti"
tp="1" # do timepoint1

#Create nifti directory
os.makedirs(niidir,exist_ok=True)

for tp in ["2", "3", "4"]: # do timepoints 2:4
    dcmdir = "/Volumes/data/prebiostress/data/sourcedata/dicom/timepoint"+tp

    #RL
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_RL_0*')
        datadir = ''.join(datadir)
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-rl_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
        print(cmd)
        os.system(cmd)
    #LR
    for fname in listdir_nohidden(dcmdir):
        subj = fname.split("_")[-3].split("_")[0]

        datadir = glob.glob(dcmdir+'/'+fname+'/*/DWI_MB3_DIR64_LR_0*')
        datadir = ''.join(datadir)
        cmd = "dcm2niix -z y -f sub-" +subj[1:] +"_ses-" +tp +"_acq-lr_dwi " \
        "-o " +niidir +"/sub-" +subj[1:] +"/ses-" +tp +"/dwi " +datadir
        print(cmd)
        os.system(cmd)
        

dcm2niix -z y -f sub-01_ses-2_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-01/ses-2/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A01_P02B_83227/CRANE_BREBIS_20180413_072122_500000/DWI_MB3_DIR64_RL_0003
dcm2niix -z y -f sub-02_ses-2_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-02/ses-2/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A02_P02B_83256/CRANE_BREBIS_20180413_084039_890000/DWI_MB3_DIR64_RL_0003
dcm2niix -z y -f sub-03_ses-2_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-03/ses-2/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A03_P02B_83262/CRANE_BREBIS_20180413_100328_734000/DWI_MB3_DIR64_RL_0003
dcm2niix -z y -f sub-04_ses-2_acq-rl_dwi -o /Volumes/data/prebiostress/data/sourcedata/nifti/sub-04/ses-2/dwi /Volumes/data/prebiostress/data/sourcedata/dicom/timepoint2/OV2018RN1_A04_P02B_83286/CRANE_BREBIS_20180413_111803_203000/DWI_MB3_DI

# Create Average T1 & T2 Images


In [2]:
niftipath = '/Volumes/data/prebiostress/data/sourcedata/nifti'
derivatives = "/Volumes/data/prebiostress/data/derivatives"

layout = BIDSLayout(niftipath,validate=False)

In [27]:
# do T1
for subj in layout.get_subjects():
    for ses in layout.get_sessions():
        targ = os.path.join(niftipath,"sub-"+subj,"ses-"+ses,"anat") # find path to anat folder for this subject and this session
        fpaths = glob.glob(targ+"/*T1w.nii.gz") # find all T1s
        outpath = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
        outpath = outpath+"/sub-"+subj+"_ses-"+ses+"_T1w.nii.gz"
        nT1s = len(fpaths)
        if nT1s > 1: # check there is more than one image to average
            # average all the T1s found
            cmd = "AverageImages 3 " +outpath + " 0 " +" ".join(fpaths)
            os.system(cmd)
            print("Averaged " +str(nT1s) +" images for Subj-" +subj +"_ses-" +ses)
        elif nT1s == 1: # if there was only 1 image then copy it to derivatives
            shutil.copy(" ".join(fpaths),outpath)
            print("Moved the only image for Subj-" +subj +"_ses-" +ses)
        elif nT1s == 0:
            print("No data for " +subj +"_ses-" +ses)
        

Average 3 images for Subj-01_ses-1
Average 2 images for Subj-01_ses-2
Average 2 images for Subj-01_ses-3
Average 2 images for Subj-01_ses-4
Average 3 images for Subj-02_ses-1
Average 2 images for Subj-02_ses-2
Average 2 images for Subj-02_ses-3
Average 2 images for Subj-02_ses-4
Average 3 images for Subj-03_ses-1
Average 2 images for Subj-03_ses-2
Average 2 images for Subj-03_ses-3
Average 2 images for Subj-03_ses-4
Average 2 images for Subj-04_ses-1
Average 2 images for Subj-04_ses-2
Average 2 images for Subj-04_ses-3
No data for 04_ses-4
Average 2 images for Subj-05_ses-1
Average 2 images for Subj-05_ses-2
Average 2 images for Subj-05_ses-3
Average 2 images for Subj-05_ses-4
Average 2 images for Subj-06_ses-1
Average 2 images for Subj-06_ses-2
Average 2 images for Subj-06_ses-3
Average 2 images for Subj-06_ses-4
Average 2 images for Subj-07_ses-1
Average 2 images for Subj-07_ses-2
Move the only image for Subj-07_ses-3
Average 2 images for Subj-07_ses-4
Average 2 images for Subj-08_se

In [5]:
# do T2
for subj in layout.get_subjects(): # loop on number of subjects
    for ses in layout.get_sessions(): # loop on number of sessions
        targ = os.path.join(niftipath,"sub-"+subj,"ses-"+ses,"anat") # find path to anat folder for this subject and this session
        fpaths = glob.glob(targ+"/*T2w.nii.gz") # find all T2s
        outpath = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
        outpath = outpath+"/sub-"+subj+"_ses-"+ses+"_T2w.nii.gz"
        nT2s = len(fpaths)
        if nT2s > 1: # check there is more than one image to average
            # average all the T1s found
            cmd = "AverageImages 3 " +outpath + " 0 " +" ".join(fpaths)
            os.system(cmd)
            print("Averaged " +str(nT2s) +" images for Subj-" +subj +"_ses-" +ses)
        elif nT2s == 1: # if there was only 1 image then copy it to derivatives
            shutil.copy(" ".join(fpaths),outpath)
            print("Moved the only image for Subj-" +subj +"_ses-" +ses)
        elif nT2s == 0:
            print("No data for " +subj +"_ses-" +ses)

Averaged 3 images for Subj-01_ses-1
Averaged 2 images for Subj-01_ses-2
Averaged 2 images for Subj-01_ses-3
Averaged 2 images for Subj-01_ses-4
Averaged 3 images for Subj-02_ses-1
Averaged 2 images for Subj-02_ses-2
Averaged 2 images for Subj-02_ses-3
Averaged 2 images for Subj-02_ses-4
Averaged 3 images for Subj-03_ses-1
Averaged 2 images for Subj-03_ses-2
Averaged 2 images for Subj-03_ses-3
Averaged 2 images for Subj-03_ses-4
Averaged 3 images for Subj-04_ses-1
Averaged 2 images for Subj-04_ses-2
Averaged 2 images for Subj-04_ses-3
No data for 04_ses-4
Averaged 2 images for Subj-05_ses-1
Averaged 2 images for Subj-05_ses-2
Averaged 2 images for Subj-05_ses-3
Averaged 2 images for Subj-05_ses-4
Averaged 2 images for Subj-06_ses-1
Averaged 2 images for Subj-06_ses-2
Averaged 2 images for Subj-06_ses-3
Averaged 2 images for Subj-06_ses-4
Averaged 2 images for Subj-07_ses-1
Averaged 3 images for Subj-07_ses-2
Moved the only image for Subj-07_ses-3
Averaged 2 images for Subj-07_ses-4
Aver

### Check each averaged image for problems:
* sub-02_ses-3 T1 has motion artifact - there was a power cut during acquisition. There is no T1_2NEX. Also the quality of the run-1 T1 is not good. **The only T1 used was run-2. For the 'Create T2 in T1 Space' task it was also necessary to do a Rigid registration between the T1 and T2 rather than just a reslice-identity.**
* sub-03_ses-1 T1 there is bad artifact (motion or Gibbs) in the cerebellum. It is present in all 3 runs but it is not 'identical' across the runs - **the average of all 3 is slightly better than any individual so we either use this or have no T1 for this subject/session**
* sub-04_ses-1 looked blurrier than others. From looking at the T1s there appears to be a slight missalignment in the anterior/posterior axis - **a rigid registration was ran between them and the average recreated**
* sub-15 and sub-15b - **decide what to do**


# T1 & T2 Denoising
### Check each denoised image for problems:

# Create T2 in T1 Space Images

In [14]:
derivatives = "/Volumes/data/prebiostress/data/derivatives"
layout = BIDSLayout(derivatives,validate=False)

for subj in layout.get_subjects(): # loop on number of subjects
    for ses in layout.get_sessions(): # loop on number of sessions
        targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat") # find path to anat folder for this subject and this session
        t1 = glob.glob(targ+"/*T1w_dn.nii.gz") # find denoised T2
        t2 = glob.glob(targ+"/*T2w_dn.nii.gz") # find denoised T1
        if t1 and t2:
            outpath = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat")
            outpath = outpath+"/sub-"+subj+"_ses-"+ses+"_T2w-in-T1w.nii.gz"
            cmd = "c3d " +" ".join(t1) +" " +" ".join(t2) +" -reslice-identity -o " +outpath
            os.system(cmd)

# Skull Stripping

In [15]:
%%bash
# crop the T1s and resample to 0.8mm isotropic. 
# The position/dimensions of the box were defined manually:

cd /Volumes/data/prebiostress/data/derivatives

# ses-4
c3d sub-01/ses-4/anat/sub-01_ses-4_T1w_dn.nii.gz -region 124x1x107vox 134x117x180vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-01/ses-4/anat/sub-01_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-02/ses-4/anat/sub-02_ses-4_T1w_dn.nii.gz -region 126x5x92vox 133x125x179vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-02/ses-4/anat/sub-02_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-03/ses-4/anat/sub-03_ses-4_T1w_dn.nii.gz -region 128x1x87vox 131x125x186vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-03/ses-4/anat/sub-03_ses-4_T1w_dn_crop-08-pad12.nii.gz

c3d sub-05/ses-4/anat/sub-05_ses-4_T1w_dn.nii.gz -region 128x8x100vox 127x117x180vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-05/ses-4/anat/sub-05_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-06/ses-4/anat/sub-06_ses-4_T1w_dn.nii.gz -region 131x10x95vox 124x122x171vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-06/ses-4/anat/sub-06_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-07/ses-4/anat/sub-07_ses-4_T1w_dn.nii.gz -region 127x1x90vox 129x123x176vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-07/ses-4/anat/sub-07_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-08/ses-4/anat/sub-08_ses-4_T1w_dn.nii.gz -region 128x2x104vox 129x119x177vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-08/ses-4/anat/sub-08_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-09/ses-4/anat/sub-09_ses-4_T1w_dn.nii.gz -region 131x3x90vox 127x122x180vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-09/ses-4/anat/sub-09_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-10/ses-4/anat/sub-10_ses-4_T1w_dn.nii.gz -region 125x1x105vox 133x125x176vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-10/ses-4/anat/sub-10_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-11/ses-4/anat/sub-11_ses-4_T1w_dn.nii.gz -region 123x2x99vox 133x126x179vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-11/ses-4/anat/sub-11_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-12/ses-4/anat/sub-12_ses-4_T1w_dn.nii.gz -region 133x2x103vox 128x123x172vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-12/ses-4/anat/sub-12_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-13/ses-4/anat/sub-13_ses-4_T1w_dn.nii.gz -region 123x1x93vox 133x135x176vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-13/ses-4/anat/sub-13_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-14/ses-4/anat/sub-14_ses-4_T1w_dn.nii.gz -region 128x2x106vox 131x124x180vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-14/ses-4/anat/sub-14_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-15/ses-4/anat/sub-15_ses-4_T1w_dn.nii.gz -region 127x1x101vox 133x125x174vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-15/ses-4/anat/sub-15_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-16/ses-4/anat/sub-16_ses-4_T1w_dn.nii.gz -region 127x1x106vox 130x131x177vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-16/ses-4/anat/sub-16_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-17/ses-4/anat/sub-17_ses-4_T1w_dn.nii.gz -region 127x5x105vox 130x133x175vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-17/ses-4/anat/sub-17_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-18/ses-4/anat/sub-18_ses-4_T1w_dn.nii.gz -region 128x1x104vox 134x126x173vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-18/ses-4/anat/sub-18_ses-4_T1w_dn_crop-08-pad12.nii.gz


c3d sub-21/ses-4/anat/sub-21_ses-4_T1w_dn.nii.gz -region 123x1x108vox 133x128x175vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-21/ses-4/anat/sub-21_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-22/ses-4/anat/sub-22_ses-4_T1w_dn.nii.gz -region 127x2x95vox 131x124x175vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-22/ses-4/anat/sub-22_ses-4_T1w_dn_crop-08-pad12.nii.gz
c3d sub-23/ses-4/anat/sub-23_ses-4_T1w_dn.nii.gz -region 127x1x111vox 132x115x175vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-23/ses-4/anat/sub-23_ses-4_T1w_dn_crop-08-pad12.nii.gz

# ses-3
c3d sub-01/ses-3/anat/sub-01_ses-3_T1w_dn.nii.gz -region 129x4x110vox 123x124x166vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-01/ses-3/anat/sub-01_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-02/ses-3/anat/sub-02_ses-3_T1w_dn.nii.gz -region 131x8x90vox 127x120x169vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-02/ses-3/anat/sub-02_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-03/ses-3/anat/sub-03_ses-3_T1w_dn.nii.gz -region 127x8x103vox 129x115x175vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-03/ses-3/anat/sub-03_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-04/ses-3/anat/sub-04_ses-3_T1w_dn.nii.gz -region 58x78x80vox 134x125x173vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-04/ses-3/anat/sub-04_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-05/ses-3/anat/sub-05_ses-3_T1w_dn.nii.gz -region 131x5x99vox 123x122x163vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-05/ses-3/anat/sub-05_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-06/ses-3/anat/sub-06_ses-3_T1w_dn.nii.gz -region 131x1x91vox 122x124x162vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-06/ses-3/anat/sub-06_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-07/ses-3/anat/sub-07_ses-3_T1w_dn.nii.gz -region 129x3x110vox 124x118x167vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-07/ses-3/anat/sub-07_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-08/ses-3/anat/sub-08_ses-3_T1w_dn.nii.gz -region 125x5x95vox 125x120x170vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-08/ses-3/anat/sub-08_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-09/ses-3/anat/sub-09_ses-3_T1w_dn.nii.gz -region 126x2x102vox 128x123x166vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-09/ses-3/anat/sub-09_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-10/ses-3/anat/sub-10_ses-3_T1w_dn.nii.gz -region 130x1x97vox 130x118x166vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-10/ses-3/anat/sub-10_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-11/ses-3/anat/sub-11_ses-3_T1w_dn.nii.gz -region 133x1x98vox 129x120x170vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-11/ses-3/anat/sub-11_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-12/ses-3/anat/sub-12_ses-3_T1w_dn.nii.gz -region 134x1x97vox 128x119x164vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-12/ses-3/anat/sub-12_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-13/ses-3/anat/sub-13_ses-3_T1w_dn.nii.gz -region 64x68x90vox 131x135x166vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-13/ses-3/anat/sub-13_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-14/ses-3/anat/sub-14_ses-3_T1w_dn.nii.gz -region 126x5x101vox 128x116x171vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-14/ses-3/anat/sub-14_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-15/ses-3/anat/sub-15_ses-3_T1w_dn.nii.gz -region 130x1x105vox 133x122x169vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-15/ses-3/anat/sub-15_ses-3_T1w_dn_crop-08-pad12.nii.gz

c3d sub-16/ses-3/anat/sub-16_ses-3_T1w_dn.nii.gz -region 129x1x101vox 128x120x168vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-16/ses-3/anat/sub-16_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-17/ses-3/anat/sub-17_ses-3_T1w_dn.nii.gz -region 138x5x100vox 124x116x162vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-17/ses-3/anat/sub-17_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-18/ses-3/anat/sub-18_ses-3_T1w_dn.nii.gz -region 124x1x96vox 131x120x164vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-18/ses-3/anat/sub-18_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-19/ses-3/anat/sub-19_ses-3_T1w_dn.nii.gz -region 133x1x93vox 128x119x176vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-19/ses-3/anat/sub-19_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-20/ses-3/anat/sub-20_ses-3_T1w_dn.nii.gz -region 125x1x91vox 129x118x174vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-20/ses-3/anat/sub-20_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-21/ses-3/anat/sub-21_ses-3_T1w_dn.nii.gz -region 128x6x99vox 130x128x166vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-21/ses-3/anat/sub-21_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-22/ses-3/anat/sub-22_ses-3_T1w_dn.nii.gz -region 126x3x118vox 130x122x168vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-22/ses-3/anat/sub-22_ses-3_T1w_dn_crop-08-pad12.nii.gz
c3d sub-23/ses-3/anat/sub-23_ses-3_T1w_dn.nii.gz -region 128x7x102vox 128x116x171vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-23/ses-3/anat/sub-23_ses-3_T1w_dn_crop-08-pad12.nii.gz


# ses-2
c3d sub-01/ses-2/anat/sub-01_ses-2_T1w_dn.nii.gz -region 129x4x105vox 124x116x164vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-01/ses-2/anat/sub-01_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-02/ses-2/anat/sub-02_ses-2_T1w_dn.nii.gz -region 132x46x105vox 129x115x161vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-02/ses-2/anat/sub-02_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-03/ses-2/anat/sub-03_ses-2_T1w_dn.nii.gz -region 124x7x88vox 124x115x168vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-03/ses-2/anat/sub-03_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-04/ses-2/anat/sub-04_ses-2_T1w_dn.nii.gz -region 130x45x101vox 129x114x167vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-04/ses-2/anat/sub-04_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-05/ses-2/anat/sub-05_ses-2_T1w_dn.nii.gz -region 130x49x104vox 118x114x154vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-05/ses-2/anat/sub-05_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-06/ses-2/anat/sub-06_ses-2_T1w_dn.nii.gz -region 127x11x103vox 120x115x157vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-06/ses-2/anat/sub-06_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-07/ses-2/anat/sub-07_ses-2_T1w_dn.nii.gz -region 135x37x108vox 118x112x168vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-07/ses-2/anat/sub-07_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-08/ses-2/anat/sub-08_ses-2_T1w_dn.nii.gz -region 131x15x99vox 124x114x163vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-08/ses-2/anat/sub-08_ses-2_T1w_dn_crop-08-pad12.nii.gz

c3d sub-10/ses-2/anat/sub-10_ses-2_T1w_dn.nii.gz -region 128x10x83vox 126x116x161vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-10/ses-2/anat/sub-10_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-11/ses-2/anat/sub-11_ses-2_T1w_dn.nii.gz -region 128x7x77vox 128x114x161vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-11/ses-2/anat/sub-11_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-12/ses-2/anat/sub-12_ses-2_T1w_dn.nii.gz -region 133x17x109vox 123x110x155vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-12/ses-2/anat/sub-12_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-13/ses-2/anat/sub-13_ses-2_T1w_dn.nii.gz -region 132x10x93vox 126x122x159vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-13/ses-2/anat/sub-13_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-14/ses-2/anat/sub-14_ses-2_T1w_dn.nii.gz -region 127x17x103vox 123x115x159vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-14/ses-2/anat/sub-14_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-15/ses-2/anat/sub-15_ses-2_T1w_dn.nii.gz -region 126x12x107vox 126x108x158vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-15/ses-2/anat/sub-15_ses-2_T1w_dn_crop-08-pad12.nii.gz

c3d sub-16/ses-2/anat/sub-16_ses-2_T1w_dn.nii.gz -region 129x43x106vox 128x111x162vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-16/ses-2/anat/sub-16_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-17/ses-2/anat/sub-17_ses-2_T1w_dn.nii.gz -region 131x13x111vox 123x114x157vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-17/ses-2/anat/sub-17_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-18/ses-2/anat/sub-18_ses-2_T1w_dn.nii.gz -region 136x57x111vox 126x106x153vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-18/ses-2/anat/sub-18_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-19/ses-2/anat/sub-19_ses-2_T1w_dn.nii.gz -region 133x13x103vox 123x118x161vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-19/ses-2/anat/sub-19_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-20/ses-2/anat/sub-20_ses-2_T1w_dn.nii.gz -region 133x46x115vox 123x111x164vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-20/ses-2/anat/sub-20_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-21/ses-2/anat/sub-21_ses-2_T1w_dn.nii.gz -region 132x23x107vox 126x118x159vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-21/ses-2/anat/sub-21_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-22/ses-2/anat/sub-22_ses-2_T1w_dn.nii.gz -region 135x4x97vox 120x118x158vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-22/ses-2/anat/sub-22_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-23/ses-2/anat/sub-23_ses-2_T1w_dn.nii.gz -region 126x4x95vox 122x111x159vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-23/ses-2/anat/sub-23_ses-2_T1w_dn_crop-08-pad12.nii.gz
c3d sub-24/ses-2/anat/sub-24_ses-2_T1w_dn.nii.gz -region 150x15x107vox 124x114x165vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-24/ses-2/anat/sub-24_ses-2_T1w_dn_crop-08-pad12.nii.gz

# ses-1
c3d sub-01/ses-1/anat/sub-01_ses-1_T1w_dn.nii.gz -region 69x89x114vox 115x107x145vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-01/ses-1/anat/sub-01_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-02/ses-1/anat/sub-02_ses-1_T1w_dn.nii.gz -region 67x105x107vox 119x108x145vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-02/ses-1/anat/sub-02_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-03/ses-1/anat/sub-03_ses-1_T1w_dn.nii.gz -region 68x74x100vox 117x112x148vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-03/ses-1/anat/sub-03_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-04/ses-1/anat/sub-04_ses-1_T1w_dn.nii.gz -region 130x42x78vox 126x107x142vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-04/ses-1/anat/sub-04_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-05/ses-1/anat/sub-05_ses-1_T1w_dn.nii.gz -region 124x46x93vox 113x106x139vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-05/ses-1/anat/sub-05_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-06/ses-1/anat/sub-06_ses-1_T1w_dn.nii.gz -region 135x38x107vox 112x102x138vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-06/ses-1/anat/sub-06_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-07/ses-1/anat/sub-07_ses-1_T1w_dn.nii.gz -region 137x30x94vox 112x115x139vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-07/ses-1/anat/sub-07_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-08/ses-1/anat/sub-08_ses-1_T1w_dn.nii.gz -region 129x28x102vox 116x107x144vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-08/ses-1/anat/sub-08_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-09/ses-1/anat/sub-09_ses-1_T1w_dn.nii.gz -region 138x38x97vox 111x107x145vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-09/ses-1/anat/sub-09_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-10/ses-1/anat/sub-10_ses-1_T1w_dn.nii.gz -region 134x30x96vox 120x107x137vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-10/ses-1/anat/sub-10_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-11/ses-1/anat/sub-11_ses-1_T1w_dn.nii.gz -region 143x41x101vox 118x106x142vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-11/ses-1/anat/sub-11_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-12/ses-1/anat/sub-12_ses-1_T1w_dn.nii.gz -region 136x40x111vox 116x112x140vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-12/ses-1/anat/sub-12_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-13/ses-1/anat/sub-13_ses-1_T1w_dn.nii.gz -region 140x33x83vox 115x110x143vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-13/ses-1/anat/sub-13_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-14/ses-1/anat/sub-14_ses-1_T1w_dn.nii.gz -region 132x35x116vox 115x109x141vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-14/ses-1/anat/sub-14_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-15/ses-1/anat/sub-15_ses-1_T1w_dn.nii.gz -region 134x20x112vox 118x112x139vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-15/ses-1/anat/sub-15_ses-1_T1w_dn_crop-08-pad12.nii.gz

c3d sub-16/ses-1/anat/sub-16_ses-1_T1w_dn.nii.gz -region 131x35x116vox 120x107x148vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-16/ses-1/anat/sub-16_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-17/ses-1/anat/sub-17_ses-1_T1w_dn.nii.gz -region 134x38x120vox 116x110x143vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-17/ses-1/anat/sub-17_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-18/ses-1/anat/sub-18_ses-1_T1w_dn.nii.gz -region 137x41x110vox 118x104x145vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-18/ses-1/anat/sub-18_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-19/ses-1/anat/sub-19_ses-1_T1w_dn.nii.gz -region 125x29x93vox 121x108x151vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-19/ses-1/anat/sub-19_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-20/ses-1/anat/sub-20_ses-1_T1w_dn.nii.gz -region 132x34x111vox 115x103x149vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-20/ses-1/anat/sub-20_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-21/ses-1/anat/sub-21_ses-1_T1w_dn.nii.gz -region 134x22x116vox 122x109x145vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-21/ses-1/anat/sub-21_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-22/ses-1/anat/sub-22_ses-1_T1w_dn.nii.gz -region 136x33x94vox 117x111x149vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-22/ses-1/anat/sub-22_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-23/ses-1/anat/sub-23_ses-1_T1w_dn.nii.gz -region 136x26x93vox 118x105x151vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-23/ses-1/anat/sub-23_ses-1_T1w_dn_crop-08-pad12.nii.gz
c3d sub-24/ses-1/anat/sub-24_ses-1_T1w_dn.nii.gz -region 129x22x99vox 117x110x151vox -resample-mm 0.8x0.8x0.8mm -pad 12x12x12vox 12x12x12vox 0 -o sub-24/ses-1/anat/sub-24_ses-1_T1w_dn_crop-08-pad12.nii.gz



/Volumes/data/prebiostress/data/derivatives


In [ ]:
ovin = "/Volumes/data/sheep-brain-atlases/"
t1s = glob.glob(ovin+"*_d_crop_08_pad12.nii.gz")
masks = glob.glob(ovin+"*_d_crop_08_brainmask_pad12.nii.gz")
atlases = ""
for t1, mask in zip(t1s, masks):
    atlases += ("-g " +t1 +" -l " +mask +" ")


derivatives = "/Volumes/data/prebiostress/data/derivatives"
layout = BIDSLayout(derivatives,validate=False)

for subj in layout.get_subjects():
    for ses in layout.get_sessions():
        targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat") # find path to anat folder for this subject and this session
        output = os.path.join(targ,"brainmask-allTemps/")
        os.makedirs(output,exist_ok=True)
        
        tpath = glob.glob(targ+"/*T1w_dn_crop-08-pad12.nii.gz") # find all T1s   
        tpath = "".join(tpath)
        
        if tpath:
            print(subj)
            print(ses)
            cmd = "antsJointLabelFusion.sh -d 3 -t " +tpath +" -o " +output +" " +atlases +"-y sr -c 2 -j 8 -q 0 > " +output+"/log.txt" 
            os.system(cmd)
            

01
01
1
02
02
1
03
03
1
04
04
1
05
05
1
06
06
1
07
07
1


# Calculate Intracranial Volume

# Bias Correction

# DWI Preprocessing

# Myelin Mapping

# Compare Hippocampus Volume

# Compare Hippocampus Connectivity